In [1]:
import ib_insync as ib
from ib_insync import *
from util import order_util
import pandas as pd
import numpy as np
from util import dt_util
import ta
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import pytz
util.startLoop()

ib = IB()

In [2]:


ib = IB()

In [3]:
ib.connect(host='127.0.0.1', port=7497, clientId=11)

<IB connected to 127.0.0.1:7497 clientId=11>

In [4]:
contract = Stock('SNAP', 'SMART', 'USD')

bars = ib.reqHistoricalData(
    contract,
    endDateTime='', # time.strftime('%Y%m%d %H:%M:%S'),
    durationStr='3600 S',
    barSizeSetting='1 min',
    whatToShow='TRADES',
    useRTH=True,
    formatDate=1,
    keepUpToDate=True
    )

df = util.df(bars)
print(df['close'])
df.head()

0     47.85
1     47.90
2     47.91
3     47.86
4     47.90
      ...  
56    47.96
57    47.91
58    47.94
59    48.00
60    48.05
Name: close, Length: 61, dtype: float64


,date,open,high,low,close,volume,average,barCount
0,2021-12-02 20:37:00,47.82,47.87,47.80,47.85,109.58,47.846,73
1,2021-12-02 20:38:00,47.87,47.90,47.86,47.90,113.66,47.876,67
2,2021-12-02 20:39:00,47.90,47.93,47.89,47.91,60.73,47.911,46
3,2021-12-02 20:40:00,47.92,47.93,47.83,47.86,162.56,47.873,91
4,2021-12-02 20:41:00,47.86,47.90,47.85,47.90,279.95,47.873,125


In [5]:
closes = []
for bar in bars:
    closes.append(bar.close)
    close_array = pd.Series(np.asarray(closes))
    print (close_array)

0    47.85
dtype: float64
0    47.85
1    47.90
dtype: float64
0    47.85
1    47.90
2    47.91
dtype: float64
0    47.85
1    47.90
2    47.91
3    47.86
dtype: float64
0    47.85
1    47.90
2    47.91
3    47.86
4    47.90
dtype: float64
0    47.85
1    47.90
2    47.91
3    47.86
4    47.90
5    47.86
dtype: float64
0    47.85
1    47.90
2    47.91
3    47.86
4    47.90
5    47.86
6    47.86
dtype: float64
0    47.85
1    47.90
2    47.91
3    47.86
4    47.90
5    47.86
6    47.86
7    47.85
dtype: float64
0    47.85
1    47.90
2    47.91
3    47.86
4    47.90
5    47.86
6    47.86
7    47.85
8    47.81
dtype: float64
0    47.85
1    47.90
2    47.91
3    47.86
4    47.90
5    47.86
6    47.86
7    47.85
8    47.81
9    47.90
dtype: float64
0     47.85
1     47.90
2     47.91
3     47.86
4     47.90
5     47.86
6     47.86
7     47.85
8     47.81
9     47.90
10    47.80
dtype: float64
0     47.85
1     47.90
2     47.91
3     47.86
4     47.90
5     47.86
6     47.86
7     47.85
8 

In [6]:
def realtimeBar(self, time, open_, high, low, close,volume, wap, count):
        super().realtimeBar(time, open_, high, low, close, volume, wap, count)
        try:
            onBarUpdate(time, open_, high, low, close, volume, wap, count)
        except Exception as e:
            print(e)
def error(self, id, errorCode, errorMsg):
    print(errorCode)
    print(errorMsg)

In [7]:
def onBarUpdate(self, hasNewBar, realtime):
    
    
    bartime = datetime.strptime(bar.date,"%Y%m%d %H:%M:%S").astimezone(pytz.timezone("America/New_York"))
    minutes_diff = (bartime-initialbartime).total_seconds() / 60.0
    currentBar.date = bartime
    lastBar = bars[len(bars) - 1]
    print (lastbar)
    # i = int(lambda i: i + 1 )

    if (minutes_diff > 0 and math.floor(minutes_diff) % self.barsize == 0):
        initialbartime = bartime

        # On Bar Close
        opens = []
        closes = []
        highs = []
        lows = []
        volumes = []
        for bar in bars:
            opens.append(bar.open) 
            closes.append(bar.close)
            highs.append(bar.high)
            lows.append(bar.low)
            volumes.append(bar.volume)
        close_array = pd.Series(np.asarray(closes))
        high_array = pd.Series(np.asarray(highs))
        low_array = pd.Series(np.asarray(lows))

        # Calculate Higher Highs and Lows # lastbar.{}
        lastLow = bars[len(bars) - 1].low
        lastHigh = bars[len(bars) - 1].high
        lastClose = bars[len(bars) - 1].close
        lastOpen = bars[len(bars) - 1].open
        lastVolume = bars[len(bars) - 1].volume

        # SMA
        sma = ta.trend.sma_indicator(close_array, 20, True)
        rsi = ta.momentum.rsi(close_array, 20, True)
        macd = ta.trend.macd(close_array, 26, 12, 9)
        print("SMA : " + str(sma[len(sma) - 1]))
        print("RSI : " + str(rsi[len(rsi) - 1]))
        print("MACD : " + str(macd[len(macd) - 1]))
        print("Close : " + str(bars[len(bars) - 1].close))
        print('lastHigh :' + str(bars[len(bars) - 2].high))
        print('lastLow :' + str(bars[len(bars) - 2].low))
        print("_____")

        # RSI > 40, < 30
        #self.rsi = talib.RSI(self.bars[0].close)
        #print('RSI : ' + self.rsi)
        # self.stochrsi = ta.momentum.StochasticOscillator(self.close_array, self.rsi_window,
        # self.smooth_window1, self.smooth_window2, True)

        # MACD
        # self.macd = ta.trend.MACD(self.close_array,self.macd_window_slow, self.macd_window_fast,
        #                          self.macd_window_sign,self.fillna)

        # Check Criteria
        # Entry - If we have a higher high, a higher low and we cross the 50 SMA - Buy

        '''plt.close()
        plot = util.barplot(bars)
        clear_output(wait=True)
        display(plot)'''

        if (bar.close > bars[len(bars) - 1].low
            and bars[len(bars) - 1].low > bars[len(bars) - 2].low
            and bar.close > str(self.sma[len(self.sma) - 1])
            and bars[len(bars) - 1].close > str(self.sma[len(self.sma) - 2])):
                print ('Strategy True - Buy')
        print ('Done')
        
    if (self.currentBar.open == 0):
            self.currentBar.open = bar.open
        if (self.currentBar.high == 0 or bar.high > self.currentBar.high):
            self.currentBar.high = bar.high
        if (self.currentBar.low == 0 or bar.low < self.currentBar.low):
            self.currentBar.low = bar.low

bars.updateEvent += onBarUpdate

ib.sleep(10)

In [8]:
ib.cancelHistoricalData(bars)
ib.disconnect()